In [2]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [6]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [7]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [74]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token != "SCREEN_NAME"]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [107]:
import random
import pandas as pd

text_data = []
corpus = pd.read_csv('datasets/data.csv', delimiter="|")
for line in corpus['text']:
    tokens = prepare_text_for_lda(line)
    #if random.random() > .99:
    #    print(tokens)
    #    text_data.append(tokens)
    text_data.append(tokens)

In [108]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [109]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [110]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [111]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

Output streaming troncato alle ultime 5000 righe.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use

In [102]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.022*"amazon" + 0.011*"serverless" + 0.011*"feature" + 0.011*"series"')
(1, '0.022*"machine" + 0.015*"serena" + 0.015*"learning" + 0.008*"looking"')
(2, '0.019*"social" + 0.019*"amaze" + 0.010*"looking" + 0.010*"system"')
(3, '0.019*"using" + 0.013*"first" + 0.013*"learn" + 0.013*"years"')
(4, '0.013*"start" + 0.013*"cloud" + 0.013*"focus" + 0.013*"community"')
(5, '0.017*"today" + 0.017*"codenewbie" + 0.017*"100daysofcode" + 0.009*"great"')
(6, '0.016*"linux" + 0.016*"need" + 0.016*"group" + 0.016*"people"')
(7, '0.017*"night" + 0.017*"microservices" + 0.009*"almost" + 0.009*"think"')
(8, '0.024*"community" + 0.016*"cloud" + 0.016*"ireland" + 0.009*"automation"')
(9, '0.012*"first" + 0.012*"world" + 0.012*"sonic" + 0.012*"access"')


In [156]:
from pprint import pprint

# Predict on a custom phrase
print("\n")

count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

TEST = pd.read_csv('datasets/data-balubor.csv', usecols= ['user','text'], delimiter="|")
for i, new_doc in enumerate(TEST['text']):
  print("==========================================================")
  pprint(TEST['text'][i])
  print("============================ : ===========================")
  new_doc = prepare_text_for_lda(new_doc)
  new_doc_bow = dictionary.doc2bow(new_doc)

  max = 0
  cluster = None

  for index, topic in enumerate(ldamodel.get_document_topics(new_doc_bow)):
    if topic[1] > max:
      max = topic[1]
      cluster = topics[topic[0]][1]

  for topic in topics:
    if topic[1] == cluster:
      count[topic[0]] += 1

  print("{} {}".format(max, cluster))


print(count)
maxc = 0
maxi = None
for idx, c in enumerate(count):
  if c > maxc:
    maxi = idx

print(topics)



("Don't know you, but I'm striving to keep learning about #MachineLearning! "
 'Good article! https://t.co/3lNgOhAejA')
============================ : ===========================
9
0.41999760270118713 0.012*"first" + 0.012*"world" + 0.012*"sonic" + 0.012*"access"
("Totally agree on that, I'd also suggest @discord as a great tool for "
 'developing and maintaining a community!\n'
 '\n'
 'Great… https://t.co/RtvWSrC2D8')
============================ : ===========================
0
0.39357298612594604 0.022*"amazon" + 0.011*"serverless" + 0.011*"feature" + 0.011*"series"
('Imagine this for #MachineLearning custom workloads. Very useful '
 'https://t.co/Mo5fbcW320')
============================ : ===========================
9
0.3499993085861206 0.012*"first" + 0.012*"world" + 0.012*"sonic" + 0.012*"access"
('An interesting how to use #Terraform on complex architecture. '
 'https://t.co/9tyGB6yuwb')
============================ : ===========================
9
0.38268545269966125 0.012*"fi